# Import Libraries

In [1]:
import spacy
import os
import json
import pandas as pd
import pickle
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload
from bs4 import BeautifulSoup
import requests
import random
from spacy.training import Example
import io
import re
import datetime

# Data Collection

--------

In [4]:
# if csv exists, load it
if os.path.exists('../clean_data/medications.csv'):
    medications = pd.read_csv('../clean_data/medications.csv')
# otherwise, scrape the data
else:
    url = "https://healthy.kaiserpermanente.org/health-wellness/drug-encyclopedia."

    medications = []

    # iterate from 'a' to 'z'
    for letter in range(97, 123):
        url = url + chr(letter)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        for li in soup.select(".drug-column-4"):
            medications.append(li.text)
        
        url = "https://healthy.kaiserpermanente.org/health-wellness/drug-encyclopedia."

    # split by new line
    medications = [medication.split('\n') for medication in medications]
    # flatten and remove empty strings
    medications = [medication for sublist in medications for medication in sublist if medication != '']
    medications = pd.DataFrame(medications, columns=['medication'])
    medications.to_csv('../clean_data/medications.csv', index=False)


print(medications)

        medication
0    acetaminophen
1        acyclovir
2         Adderall
3        albuterol
4      alendronate
..             ...
375    Zithromycin
376         Zoloft
377       Zolpidem
378        Zovirax
379         Zyrtec

[380 rows x 1 columns]


380 medication names to randomize NER data

In [5]:
# Define the scope of the application
SCOPES = ['https://www.googleapis.com/auth/drive']
# Function to authenticate and create the service
def create_service():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    service = build('drive', 'v3', credentials=creds)
    return service

# Function to list files in a given folder ID
def list_files_in_folder(service, folder_id):
    results = service.files().list(q=f"'{folder_id}' in parents", fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])
    return items

# Function to download a file
def download_or_export_file(service, file_id, file_name, mime_type):
    try:
        # Check if the file is a Google Doc by its MIME type
        if mime_type.startswith('application/vnd.google-apps.'):
            # Define export MIME type for Google Docs (e.g., 'application/pdf' for Google Docs)
            if mime_type == 'application/vnd.google-apps.document':
                export_mime_type = 'application/pdf'
                file_name += '.pdf'  # Append appropriate file extension
            elif mime_type == 'application/vnd.google-apps.spreadsheet':
                export_mime_type = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
                file_name += '.xlsx'  # Append appropriate file extension
            elif mime_type == 'application/vnd.google-apps.presentation':
                export_mime_type = 'application/vnd.openxmlformats-officedocument.presentationml.presentation'
                file_name += '.pptx'  # Append appropriate file extension
            else:
                # Default to PDF for other Google Apps documents
                export_mime_type = 'application/pdf'
                file_name += '.pdf'
            
            request = service.files().export_media(fileId=file_id, mimeType=export_mime_type)
        else:
            # For binary files, use the get_media method
            request = service.files().get_media(fileId=file_id)
        
        # Perform the download or export
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}%.")
        
        # Write the file's contents to a local file
        with open(file_name, 'wb') as f:
            f.write(fh.getbuffer())
        print(f"File '{file_name}' downloaded successfully.")
    
    except Exception as e:
        print(f"An error occurred: {e}")

def find_folders_by_name(service, folder_name):
    """Find folders by name and return their IDs."""
    query = f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}'"
    response = service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
    return response.get('files', [])

def find_subfolder_id(service, parent_folder_id, subfolder_name):
    """Find a specific subfolder within a parent folder."""
    query = f"'{parent_folder_id}' in parents and mimeType='application/vnd.google-apps.folder' and name='{subfolder_name}'"
    response = service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
    files = response.get('files', [])
    if files:
        return files[0]['id']  # Return the ID of the first matching subfolder
    return None

def download_txt_files_from_folder(service, folder_id):
    """Download all .txt files from a specified folder."""
    query = f"'{folder_id}' in parents and mimeType='text/plain'"
    response = service.files().list(q=query, spaces='drive', fields='files(id, name, mimeType)').execute()
    files = response.get('files', [])
    for file in files:
        print(f"Downloading/exporting {file['name']}...")
        download_or_export_file(service, file['id'], file['name'], file['mimeType'])

Toy diagnosis data. Might look into a better source, but this will work for now

In [6]:
# again, if csv exists, load it
if os.path.exists('../clean_data/diagnoses.csv'):
    diagnoses = pd.read_csv('../clean_data/diagnoses.csv')
# otherwise, download it from Google Drive
else:
    service = create_service()  # Assume this is implemented as shown before
    top_level_folder_names = ['Base-Game', 'Mod-Diagnoses']

    for folder_name in top_level_folder_names:
        folders = find_folders_by_name(service, folder_name)
        for folder in folders:
            dept_diagnoses_folder_id = find_subfolder_id(service, folder['id'], 'Dept-Diagnoses')
            if dept_diagnoses_folder_id:
                download_txt_files_from_folder(service, dept_diagnoses_folder_id)
    
    diagnoses = []
    # loop through all files and extract the text following '##' (diagnosis names)
    for file in os.listdir('../raw_data/diagnoses'):
        with open(f'../raw_data/diagnoses/{file}', 'r') as f:
            for line in f:
                if '##' in line:
                    diagnoses.append(line.split('##')[1].strip())
    
    diagnoses = pd.DataFrame(diagnoses, columns=['diagnosis']).to_csv('../clean_data/diagnoses.csv', index=False)

print(diagnoses)

                                 diagnosis
0                  Acute Myeloid Leukaemia
1     Adrenocortical Carcinoma (Localised)
2    Adrenocortical Carcinoma (Metastatic)
3      Adrenocortical Carcinoma (Regional)
4                             ALL (B Cell)
..                                     ...
629                           Typhus Fever
630                           Valley Fever
631                        West Nile Fever
632                           Yellow Fever
633                             Zika Fever

[634 rows x 1 columns]


634 diagnoses to randomize NER training data

Some dosages are nonsensical, but will hopefully allow the model to generalize well

In [7]:
# randomly generate dosage data
dosages = []
units = ['mg', 'g', 'mL', 'L']
concat_every = 5
frequency = ['twice daily', 'once daily', 'as needed']

for hour in range(48):
    frequency.append(f'every {hour} hours')

for i in range(300):
    dosage = str(random.choice(range(5, 1001, 5))) + f' {random.choice(units)}'
    if i % concat_every == 0:
        dosage += ' ' + random.choice(frequency)
    dosages.append(dosage)

print(dosages)

['930 mL every 9 hours', '830 mg', '290 mg', '565 L', '255 L', '645 g every 20 hours', '270 g', '360 mg', '805 g', '1000 mL', '515 g every 21 hours', '925 L', '540 L', '180 mL', '80 mL', '370 mg every 16 hours', '350 L', '415 mg', '905 mL', '70 g', '875 mg every 42 hours', '285 mg', '475 g', '845 L', '305 mL', '710 mL every 39 hours', '915 g', '390 L', '415 L', '555 mg', '810 g every 41 hours', '735 g', '225 g', '685 g', '540 L', '950 mg as needed', '155 mL', '670 g', '295 g', '635 mg', '125 L every 29 hours', '635 L', '735 mg', '60 mL', '985 mL', '360 mL every 40 hours', '540 mL', '890 mL', '455 L', '125 g', '330 g every 45 hours', '415 L', '940 L', '330 mg', '445 mL', '380 mL every 5 hours', '25 mg', '85 mL', '45 g', '145 L', '840 L every 19 hours', '425 mg', '310 g', '500 L', '330 mg', '760 L every 10 hours', '125 mL', '275 g', '350 L', '900 L', '925 mg every 31 hours', '705 L', '625 g', '500 L', '590 g', '830 L as needed', '555 mL', '520 mg', '685 mg', '865 mg', '975 mg once daily'

Tests

In [8]:
# if csv exists, load it
if os.path.exists('../clean_data/tests.csv'):
    tests = pd.read_csv('../clean_data/tests.csv')
# otherwise, scrape the data
else:
    url = "https://medlineplus.gov/lab-tests/"

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    tests = []
    # select all uorderdered lists with class 'withident breaklist'
    for item in soup.select(".withident.breaklist"):
        tests.append(item.text)

    tests = pd.DataFrame(tests, columns=['test'])
    tests['test'] = tests['test'].str.split('\n')
    tests = tests.explode('test')
    tests = tests[tests['test'] != '']
    tests.to_csv('../clean_data/tests.csv', index=False)

Symptoms

In [9]:
# if csv exists, load it
if os.path.exists('../clean_data/symptoms.csv'):
    symptoms = pd.read_csv('../clean_data/symptoms.csv')
else:
    symptoms = []
    pattern = r"\+\s(.+?)\s\(\d+% of cases \| .+?\)"

    # loop through all files and extract the text following '##' (diagnosis names)
    for file in os.listdir('../raw_data/diagnoses'):
        with open(f'../raw_data/diagnoses/{file}', 'r') as f:
            for line in f:
                match = re.search(pattern, line)
                if match:
                    symptoms.append(match.group(1))

    symptoms = pd.DataFrame(symptoms, columns=['symptom'])
    # make unique
    symptoms = symptoms.drop_duplicates()
    symptoms.to_csv('../clean_data/symptoms.csv', index=False)

Dates

In [10]:
dates = []

# generate dates in the format "Month Day"
for i in range(1, 13):
    for j in range(1, 29):
        # year is arbitrary
        date = datetime.date(2021, i, j)
        full_date = f"{date.strftime('%B')} {date.strftime('%d')}"
        dates.append(full_date)

print(dates)

['January 01', 'January 02', 'January 03', 'January 04', 'January 05', 'January 06', 'January 07', 'January 08', 'January 09', 'January 10', 'January 11', 'January 12', 'January 13', 'January 14', 'January 15', 'January 16', 'January 17', 'January 18', 'January 19', 'January 20', 'January 21', 'January 22', 'January 23', 'January 24', 'January 25', 'January 26', 'January 27', 'January 28', 'February 01', 'February 02', 'February 03', 'February 04', 'February 05', 'February 06', 'February 07', 'February 08', 'February 09', 'February 10', 'February 11', 'February 12', 'February 13', 'February 14', 'February 15', 'February 16', 'February 17', 'February 18', 'February 19', 'February 20', 'February 21', 'February 22', 'February 23', 'February 24', 'February 25', 'February 26', 'February 27', 'February 28', 'March 01', 'March 02', 'March 03', 'March 04', 'March 05', 'March 06', 'March 07', 'March 08', 'March 09', 'March 10', 'March 11', 'March 12', 'March 13', 'March 14', 'March 15', 'March 

Times

In [11]:
times = []
# generate times in the format "Hour AM/PM"
for i in range(1, 13):
    for j in ["AM", "PM"]:
        time = f"{i} {j}"
        times.append(time)

Body parts

Hard-coding for now. Will change

In [12]:
anatomies = ["left femur", "right knee", "abdominal region", "left lung", "right lung", "left kidney", "right kidney",
             "left eye", "right eye", "left ear", "right ear", "left hand", "right hand", "left foot", "right foot",
             "left arm", "right arm", "left leg", "right leg", "left shoulder", "right shoulder", "left hip", "right hip",
             "left elbow", "right elbow", "left wrist", "right wrist", "left ankle", "right ankle", "left toe", "right toe",
             "left finger", "right finger", "left thumb", "right thumb", "left nostril", "right nostril", "left cheek", "right cheek",
             "left temple", "right temple", "left jaw", "right jaw", "left chin", "right chin", "left neck", "right neck", "left collarbone",
             "right collarbone", "left rib", "right rib", "left hip bone", "right hip bone", "left thigh", "right thigh", "left calf",
             "right calf", "left shin", "right shin", "left heel", "right heel", "left sole", "right sole", "left toe", "right toe",
             "left finger", "right finger", "left thumb", "right thumb", "left palm", "right palm", "left wrist", "right wrist", "left forearm",
             "right forearm", "left bicep", "right bicep", "left tricep", "right tricep", "left shoulder", "right shoulder", "left chest", "right chest",
             "left breast", "right breast", "left nipple", "right nipple", "left rib", "right rib", "left abdomen", "right abdomen", "left hip",
             "right hip", "left groin", "right groin", "left thigh", "right thigh", "left knee", "right knee", "left shin", "right shin", "left calf",
             "right calf", "left ankle", "right ankle", "left foot", "right foot", "left toe", "right toe", "left finger", "right finger", "left thumb",
             "right thumb", "left hand", "right hand", "left wrist", "right wrist", "left forearm", "right forearm", "left elbow", "right elbow",
             "left upper arm", "right upper arm", "left shoulder", "heart", "liver", "stomach", "intestines", "pancreas", "spleen", "bladder", "esophagus"]

len(anatomies)

132

In [13]:
# make unique
anatomies = list(set(anatomies))
len(anatomies)

92

-------------------

# Data Processing

---------

ToDo: Need to combine logic of next two cells to only generate a train_data.csv instead of two files

In [14]:
# Function to generate a random date
def generate_example():
    output = {}
    diagnosis = random.choice(diagnoses['diagnosis'].values)
    medication = random.choice(medications['medication'].values)
    dosage = random.choice(dosages)
    test_name = random.choice(tests['test'].values)
    symptom = random.choice(symptoms['symptom'].values)
    body_part = random.choice(anatomies)
    time = random.choice(times)
    date = random.choice(dates)

    choices = [diagnosis, medication, dosage, test_name, symptom, body_part, time, date]
    choice_map = ['diagnosis', 'medication', 'dosage', 'test_name', 'symptom', 'body_part', 'time', 'date']
    entities = []

    text_elements = [
        f"The patient was diagnosed with {diagnosis} last year.",
        f"He has been prescribed {medication} {dosage}.",
        f"A follow-up appointment is scheduled for {time} on {date}.",
        f"{test_name} measurements indicate {diagnosis}.",
        f"The {test_name} revealed a {diagnosis} in the {body_part}.",
        f"Patient presents with {symptom}.",
        f"Prescribe {dosage} of {medication} for pain relief.",
        f"The {test_name} shows normal {body_part} function.",
        f"She mentioned an allergy to {medication}.",
        f"Examine the {symptom} in the patient's {body_part}.",
    ]
    text = random.choice(text_elements)
    # Simplified entity recognition (just a placeholder, not accurate)
    for index, choice in enumerate(choices):
        if text.find(choice) != -1:
            entities.append({"start": text.find(choice), "end": text.find(choice) + len(choice), "label": f"{choice_map[index]}"})
    
    output["text"] = text
    output["entities"] = entities
    return output

# Generate 1000 examples
train_data = [generate_example() for _ in range(1000)]

# Save the examples to a JSON file
file_path = '../clean_data/train_data.json'
with open(file_path, 'w') as file:
    json.dump(train_data, file, indent=2)

In [15]:
# Load JSON data
with open('../clean_data/train_data.json', 'r') as file:
    data = json.load(file)

# Convert the JSON data to the desired format
formatted_data = []
for item in data:
    text = item['text']
    entities = []
    for entity in item['entities']:
        start = entity['start']
        end = entity['end']
        label = entity['label']
        entities.append((start, end, label.upper()))  # Convert label to uppercase as shown in the example
    formatted_data.append((text, {"entities": entities}))

# Convert to DataFrame for easy CSV saving
df = pd.DataFrame(formatted_data, columns=['Text', 'Entities'])

# Save to CSV
df.to_csv('../clean_data/formatted_train_data.csv', index=False)

In [19]:
entity_labels = set()
for text, annotations in formatted_data:
    for entity in annotations['entities']:
        entity_labels.add(entity[2])

print(entity_labels)

{'DIAGNOSIS', 'MEDICATION', 'DATE', 'TEST_NAME', 'BODY_PART', 'DOSAGE', 'SYMPTOM', 'TIME'}


In [21]:
if os.path.exists('../clean_data/models/ner_model'):
    nlp = spacy.load('../clean_data/models/ner_model')
else:
    # Load a blank model
    nlp = spacy.blank('en')

    # Add the NER pipeline if not already present
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe('ner')
    else:
        ner = nlp.get_pipe('ner')

    # Add entity labels to the model
    for entity in entity_labels:
        ner.add_label(entity)

    # Train model
    optimizer = nlp.begin_training()
    for itn in range(10):  # Number of training iterations
        for text, annotations in formatted_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.5, sgd=optimizer)

    # Save the model
    nlp.to_disk('../clean_data/models/ner_model')

Will need more rigorous testing/improvement im sure, but fine for now

In [23]:
# Example text
text = "Patient was administered 500mg of Ibuprofen."

# Process the text
doc = nlp(text)

# Iterate over the predicted entities
for ent in doc.ents:
    print(ent.text, ent.label_)

500mg DOSAGE
Ibuprofen MEDICATION
